In [8]:
import csv 
import pandas as pd 
import os 
from math import sqrt 
 
path = '/Users/jaredmaslin/Desktop/kaggle_forest_cover/' 

 
from time import time 
t0 = time() 

 
print "Let's grab for our ingredients to start making a delicious dish!" 
train_data = pd.read_csv(os.path.join(path,'train.csv'), header=0) 
train_labels = train_data['Cover_Type'].as_matrix() 
train_id = train_data['Id'].as_matrix() 
train_data.drop(['Cover_Type', 'Id'], axis=1, inplace=True) 
test_data = pd.read_csv(os.path.join(path,'test.csv'), header=0) 
test_id = test_data['Id'].as_matrix() 
test_data.drop(['Id'], axis=1, inplace=True)  

 
print "Now, let's use some of our ingredients to create a wonderful mix!" 

#Azimuth Adjustment
def adjust(x): 
    if x+180>360: 
        return x-180 
    else: 
        return x+180 

#Feature Engineering
train_data['Aspect2'] = train_data.Aspect.map(adjust) 
test_data['Aspect2'] = test_data.Aspect.map(adjust) 


train_data['Above_Sea_Level'] = train_data.Vertical_Distance_To_Hydrology < 0 
test_data['Above_Sea_Level'] = test_data.Vertical_Distance_To_Hydrology < 0 
     

train_data['Vertical_To_Water'] = train_data.Elevation-train_data.Vertical_Distance_To_Hydrology 
test_data['Vertical_To_Water'] = test_data.Elevation-test_data.Vertical_Distance_To_Hydrology 

train_data['Horizontal_To_Water'] = train_data.Elevation-train_data.Horizontal_Distance_To_Hydrology*0.2 
test_data['Horizontal_To_Water'] = test_data.Elevation-test_data.Horizontal_Distance_To_Hydrology*0.2 
      

train_data['Pythagorean_To_Water'] = (train_data['Horizontal_Distance_To_Hydrology']**2+train_data['Vertical_Distance_To_Hydrology']**2)**0.5 
test_data['Pythagorean_To_Water'] = (test_data['Horizontal_Distance_To_Hydrology']**2+test_data['Vertical_Distance_To_Hydrology']**2)**0.5 

 
train_data['Water_And_Fire'] = train_data['Horizontal_Distance_To_Hydrology']+train_data['Horizontal_Distance_To_Fire_Points'] 
test_data['Water_And_Fire'] = test_data['Horizontal_Distance_To_Hydrology']+test_data['Horizontal_Distance_To_Fire_Points'] 

 
train_data['Water_Less_Fire'] = abs(train_data['Horizontal_Distance_To_Hydrology']-train_data['Horizontal_Distance_To_Fire_Points']) 
test_data['Water_Less_Fire'] = abs(test_data['Horizontal_Distance_To_Hydrology']-test_data['Horizontal_Distance_To_Fire_Points']) 
 
#Currently hurts the score
#train_frame['Shade_Overall_Average'] = abs(train_frame['Hillshade_9am']+train_frame['Hillshade_Noon']+train_frame['Hillshade_3pm'])/3
#test_frame['Shade_Overall_Average'] = abs(test_frame['Hillshade_9am']+test_frame['Hillshade_Noon']+test_frame['Hillshade_3pm'])/3
    
train_data['Water_And_Roadway'] = abs(train_data['Horizontal_Distance_To_Hydrology']+train_data['Horizontal_Distance_To_Roadways']) 
test_data['Water_And_Roadway'] = abs(test_data['Horizontal_Distance_To_Hydrology']+test_data['Horizontal_Distance_To_Roadways']) 
 
train_data['Water_Less_Roadway'] = abs(train_data['Horizontal_Distance_To_Hydrology']-train_data['Horizontal_Distance_To_Roadways']) 
test_data['Water_Less_Roadway'] = abs(test_data['Horizontal_Distance_To_Hydrology']-test_data['Horizontal_Distance_To_Roadways']) 
 
train_data['Fire_And_Roadway'] = abs(train_data['Horizontal_Distance_To_Fire_Points']+train_data['Horizontal_Distance_To_Roadways']) 
test_data['Fire_And_Roadway'] = abs(test_data['Horizontal_Distance_To_Fire_Points']+test_data['Horizontal_Distance_To_Roadways']) 

train_data['Fire_Less_Roadway'] = abs(train_data['Horizontal_Distance_To_Fire_Points']-train_data['Horizontal_Distance_To_Roadways']) 
test_data['Fire_Less_Roadway'] = abs(test_data['Horizontal_Distance_To_Fire_Points']-test_data['Horizontal_Distance_To_Roadways']) 


train_matrix = train_data.as_matrix() 
test_X = test_data.as_matrix() 
n_features = int(sqrt(train_matrix.shape[1])) 
 
print "With our best magnifying glass, lets find the right combination of ingredients and get ready to bake!"  

from sklearn.ensemble import ExtraTreesClassifier 

clf = ExtraTreesClassifier(n_estimators=875) 
clf = clf.fit(train_matrix, train_labels)
 
print 'With the oven preheated and our ingredients ready, pop it in the oven!' 
predictions = clf.predict(test_X).astype(int) 
print predictions.shape 
 
print "Just need to cut our delicious baked good into slices..." 
prediction_write = open(os.path.join(path,"sampleSubmission2.csv"), "wb") 
open_file_object = csv.writer(prediction_write) 
open_file_object.writerow(["Id","Cover_Type"]) 
open_file_object.writerows(zip(test_id, predictions)) 
prediction_write.close() 
print 'Ready to eat! (Be very careful! Contents hot!)' 

print 'Wait... how long did that take?'
print("... in %0.3fs" % (time() - t0)) 

Let's grab for our ingredients to start making a delicious dish!
Now, let's use some of our ingredients to create a wonderful mix!
With our best magnifying glass, lets find the right combination of ingredients and get ready to bake!
With the oven preheated and our ingredients ready, pop it in the oven!
(565892,)
Just need to cut our delicious baked good into slices...
Ready to eat! (Be very careful! Contents hot!)
Wait... how long did that take?
... in 257.236s
